In [1]:
import numpy as np
import pandas as pd

# マスター

In [323]:
format_dic = {
    "Talk (15 minutes)": 15,
    "Talk (30 minutes)": 30,
    "Talk (45 minutes)": 45
}
lang_dic = {
    "日": "ja", "英": "en"
}
cat_dic = {
    "case_study": "Project case studies",
    "web_pro": "Web programming(including web frameworks) ",
    "library": "Python libraries",
    "practice": "Best practices",
    "hardware": "extending and embedding python in hardware",
    "education": "Python in education(including science and maths)",
    "ml_ds": "Machine learning and data science",
    "gui": "GUI and games",
    "package": "Packaging",
    "fintech": "Fintech",
    "sys_ad": "System administration",
    "tools": "Programming tools",
    "business": "Business efficiency solution",
    "etc": "Anything else basically which doesn’t really fall into the types of topics above"
}
schdule_dic = {
    1: "(45分)1日目 11:25-12:10",
    2: "(45分)1日目 13:40-14:25",
    3: "(30分)1日目 14:40-15:10",
    4: "(15分)1日目 16:00-16:15",
    5: "(15分)1日目 16:30-16:45",
    6: "(45分)2日目 11:15-12:00",
    7: "(30分)2日目 13:30-14:00",
    8: "(30分)2日目 14:15-14:45",
    9: "(15分)2日目 15:45-16:00",
    10: "(15分)2日目 16:15-16:30"
}
room_dic = {
    1: "Room A+B",
    2: "Small Exhibition Hall",
    3: "Convention Hall UME",
    4: "Convention Hall UGUISU",
    5: "Room D"
}

# トークリスト

In [320]:
talks = pd.read_csv("talks.csv").fillna(0)
talks = talks.reset_index().rename(columns={"index": "id"})
talks['id'] = talks['id'] + 1
talks['format'] = talks['format'].map(format_dic)
talks['lang'] = talks['lang'].map(lang_dic)
talks['category'] = talks['category'].map({v: k for k, v in cat_dic.items()})
talks['desc'] = talks['id'].astype(str) + " [" + talks['lang'] + "]["  + talks['category'] + "]" + talks['name'] + ":" + talks['title']
talk_dic = talks.set_index('id')['desc'].to_dict()
talks.head()

,id,name,format,lang,category,title,request,desc
0,1,Kazuki Higashiguchi,15,ja,case_study,Pythonを使ったAPIサーバー開発を始める際に整備したCIとテスト機構,0.0,1 [ja][case_study]Kazuki Higashiguchi:Pythonを使...
1,2,Yusuke Miyazaki,15,ja,web_pro,Python ウェブアプリケーションのためのプロファイラの実装,0.0,2 [ja][web_pro]Yusuke Miyazaki:Python ウェブアプリケー...
2,3,Marc-Andre Lemburg,30,en,library,PyRun - Shipping the Python 3.7 runtime in jus...,0.0,3 [en][library]Marc-Andre Lemburg:PyRun - Ship...
3,4,武山 文信,45,ja,practice,Excel と Python による社会インフラシステムの設定ファイルの自動生成,0.0,4 [ja][practice]武山 文信:Excel と Python による社会インフラ...
4,5,John Belmonte,45,en,hardware,Inside a companion robot: Productive concurren...,0.0,5 [en][hardware]John Belmonte:Inside a compani...


In [303]:
talks45 = talks.query("format == 45")
talks30 = talks.query("format == 30")
talks15 = talks.query("format == 15")

# 初期化

In [270]:
def init():
    time45 = pd.DataFrame(np.zeros((3, 5)), dtype=int, columns=range(1, 6))
    time45.iloc[1, 1] = -1
    time45.index = [1, 2, 6]
    time30 = pd.DataFrame(np.zeros((3, 5)), dtype=int, columns=range(1, 6))
    time30.index = [3, 7, 8]
    time15 = pd.DataFrame(np.zeros((4, 5)), dtype=int, columns=range(1, 6))
    time15.iloc[0:2, 1] = -1
    time15.index = [4, 5, 9, 10]
    
    return time45, time30, time15

In [271]:
time45, time30, time15 = init()

In [272]:
time45

,1,2,3,4,5
1,0,0,0,0,0
2,0,-1,0,0,0
6,0,0,0,0,0


In [273]:
time30

,1,2,3,4,5
3,0,0,0,0,0
7,0,0,0,0,0
8,0,0,0,0,0


In [274]:
time15

,1,2,3,4,5
4,0,-1,0,0,0
5,0,-1,0,0,0
9,0,0,0,0,0
10,0,0,0,0,0


In [275]:
(time45 == 0).sum().sum() + (time30 == 0).sum().sum() +  (time15 == 0).sum().sum() 

47

# 割当

## 45

In [358]:
def nth_min_cnt_keys(cnt, nth):
    sorted_cnt = sorted(set(cnt.values()))
    nth = len(sorted_cnt) -1 if nth > len(sorted_cnt) - 1 else nth
    min_values = sorted_cnt[nth]
    min_keys = [k for k, v in cnt.items() if v <= min_values]
    return min_keys

In [359]:
class AssignmentImpossible(Exception):
    pass


def assign(time, talks, random_state=None):
    # 順番をシャッフル
    if random_state:
        talks = talks.sample(frac=1, random_state=random_state)

    # 最初は選択されている回数が最小のもの。その後緩和する
    nth_lang, nth_cat = 0, 0

    # 割り当てられたtalk_ids
    assign_ids = []
    
    # カテゴリ、言語のカウント
    row_lang_cnt = {row_idx: {v: 0 for _, v in lang_dic.items()} for row_idx in time.index.tolist()}
    row_cat_cnt = {row_idx: {k: 0 for k, _ in cat_dic.items()} for row_idx in time.index.tolist()}

    for i in range(100):
        print("緩和条件 nth_lang:", nth_lang, "nth_cat:", nth_cat)
        for row_number, (row_idx, time_row) in enumerate(time.iterrows()):
            # 残りの行を計算(★で利用)
            rest_rows = time.shape[0] - row_number
            
            # 行ごとに言語、カテゴリをカウントする変数を初期化
            for col_idx, assignment in time_row.items():
    #             print("row_idx:", row_idx, "col_idx:", col_idx, "talk_id:", assignment)

                # もしすでに割り当てられていたら飛ばす
                if assignment != 0:
                    continue

                # 対象の言語、カテゴリを選択する
                # 最初は選択されている回数が最小のもの。その後緩和する
                cand_langs = nth_min_cnt_keys(row_lang_cnt[row_idx], nth_lang)
                cand_cats = nth_min_cnt_keys(row_cat_cnt[row_idx], nth_cat)
    #             print('cand_langs:', cand_langs, "cand_cats:", cand_cats)

                # まだ割り当てられていないトークを走査する
                rest = talks.query("id not in @assign_ids")
                for _, talk in rest.iterrows():
                    talk_id, lang, cat, title, request = \
                        talk['id'], talk['lang'], talk['category'], talk['title'], talk['request']

    #                 print(f"「{title}」をチェックします")
    #                 print("lang:", lang, "cat:", cat)

                    # もし希望があれば希望に当てはまっていない場合は割り当てない
                    if request and request != row_idx:
                        print("希望にあっていないので割り当てません")
                        continue
                        
                    # ★残りの言語の数が各行1つ以上ないとき割り当てない(緩和してない場合のみ)
                    if nth_lang == 0 and nth_cat == 0:
                        if rest.query("lang == @lang").shape[0] < rest_rows:
                            continue

                    if lang in cand_langs and cat in cand_cats:
                        print(f"「{title}」を割り当てます")
                        assign_ids.append(talk_id)
                        time.loc[row_idx, col_idx] = talk_id
                        row_lang_cnt[row_idx][lang] += 1
                        row_cat_cnt[row_idx][cat] += 1
                        break
                else:
                    print("割り当てられませんでした")

        # すべて割り当てられていたら終了
        if (time == 0).sum().sum() == 0:
            print("割当完了しました")
            return time, row_lang_cnt, row_cat_cnt, nth_lang, nth_cat
        # カテゴリ、言語の順番で緩和する
        if i % 2 == 0:
            nth_cat += 1
        else:
            nth_lang += 1
    else:
        msg = "割り当てに失敗しました"
        print(msg)
        raise AssignmentImpossible(msg)

In [360]:
# random_stateでversionを作成
versions = {
    1: None, 2: 1234, 3: 2345, 4: 3456, 5: 4567
}

In [368]:
dfreports = {}
for name, random_state in versions.items():
    results = []
    lang_cnts = []
    cat_cnts = []
    relaxes = []
    time45, time30, time15 = init()
    data = {
        45: (time45, talks45), 30: (time30, talks30), 15: (time15, talks15)
    }
    while True:
        try:
            for key, (time, talks) in data.items():
                result, row_lang_cnt, row_cat_cnt, nth_lang, nth_cat = assign(time, talks, random_state=random_state)
                results.append(result)
                lang_cnts.append(pd.DataFrame(row_lang_cnt))
                cat_cnts.append(pd.DataFrame(row_cat_cnt))
                relaxes.append((nth_lang, nth_cat))
        except AssignmentImpossible as e:
            print(e)
            # 再チャレンジ
            random_state += 1
        else:
            break
    
    dfresult = pd.concat(results).sort_index().replace(talk_dic)
    dfresult.columns = dfresult.columns.map(room_dic)
    dflang = pd.concat(lang_cnts, axis=1).T
    dfcat = pd.concat(cat_cnts, axis=1).T
    dfreport = pd.concat([dfresult, dflang, dfcat], axis=1)
    dfreport.index = dfreport.index.map(schdule_dic)
    dfreports[name] = dfreport

緩和条件 nth_lang: 0 nth_cat: 0
「Excel と Python による社会インフラシステムの設定ファイルの自動生成」を割り当てます
「Inside a companion robot: Productive concurrency with Trio async-await ?」を割り当てます
「KubernetesとJupyterHubで構築する機械学習eラーニングサイト」を割り当てます
希望にあっていないので割り当てません
「Automate the Boring Stuff with Slackbot」を割り当てます
「PythonとAutoML」を割り当てます
「It’s 2019 and I’m still using Python 2. Should I be worried?」を割り当てます
「Pythonで始めてみよう関数型プログラミング」を割り当てます
「Yet Another Isolation - Debian Packageと紐づく環境分離」を割り当てます
希望にあっていないので割り当てません
割り当てられませんでした
「Djangoで実践ドメイン駆動設計による実装」を割り当てます
「Modern development environments for Pythonistas」を割り当てます
「機械学習におけるハイパーパラメータ最適化の理論と実践」を割り当てます
割り当てられませんでした
割り当てられませんでした
緩和条件 nth_lang: 0 nth_cat: 1
希望にあっていないので割り当てません
割り当てられませんでした
割り当てられませんでした
割り当てられませんでした
緩和条件 nth_lang: 1 nth_cat: 1
希望にあっていないので割り当てません
「PythonとGoogle Optimization Toolsの最適化ライブラリで、「人と人の相性を考慮したシフトスケジューラ」を作ってみた。」を割り当てます
「Python Webフレームワーク比較」を割り当てます
「Pythonと便利ガジェット、サービス、ツールを使ってセンシング〜見る化してみよう」を割り当てます
割当完了しました
緩和条件 nth_lang: 0 nth_cat: 0
「PyRun - Shipping the Pyth

緩和条件 nth_lang: 12 nth_cat: 12
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 12 nth_cat: 13
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 13 nth_cat: 13
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 13 nth_cat: 14
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 14 nth_cat: 14
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 14 nth_cat: 15
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 15 nth_cat: 15
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 15 nth_cat: 16
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 16 nth_cat: 16
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 16 nth_cat: 17
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 17 nth_cat: 17
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 17 nth_cat: 18
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 18 nth_cat: 18
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 18 nth_cat: 19
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 19 nth_cat: 19
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth_lang: 19 nth_cat: 20
希望にあっていないので割り当てません
割り当てられませんでした
緩和条件 nth

割り当てられませんでした
緩和条件 nth_lang: 0 nth_cat: 1
「Getting Started with Asynchronous Python Web DevelopmentA few years ago, Python provided great tools for asynchronous programming since version 3.4 (asyncio) and version 3.5 (async/await syntax). As of 2019, the ecosystem around those new features has matured enough, it is possible to launch your new backend using the asynchronous capabilities of Python.」を割り当てます
割り当てられませんでした
割り当てられませんでした
割り当てられませんでした
割り当てられませんでした
割り当てられませんでした
割り当てられませんでした
緩和条件 nth_lang: 1 nth_cat: 1
「機械学習ライブラリのPython API作成方法」を割り当てます
「メディアが運用すべき持続可能なVTuberをつくる技術」を割り当てます
「Anaconda環境運用TIPS ?Anacondaの環境構築について知る・質問に答えられるようになる?（仮題）」を割り当てます
「Pythonを使ったAPIサーバー開発を始める際に整備したCIとテスト機構」を割り当てます
「チームメイトのためにdocstringを書こう」を割り当てます
「ListはIteratorですか？」を割り当てます
割当完了しました
緩和条件 nth_lang: 0 nth_cat: 0
希望にあっていないので割り当てません
「Pythonで始めてみよう関数型プログラミング」を割り当てます
希望にあっていないので割り当てません
「Automate the Boring Stuff with Slackbot」を割り当てます
希望にあっていないので割り当てません
「Inside a companion robot: Productive concurrency with Trio async-a

In [369]:
dfreports.keys()

dict_keys([1, 2, 3, 4, 5])

In [370]:
with pd.ExcelWriter('assignment_20190803.xlsx') as writer:
    for version, dfreport in dfreports.items():
        dfreport.to_excel(writer, sheet_name=f'version{version}')